In [ ]:
# So Colab can use the python files...
!git clone --single-branch -b develop https://github.com/muellevin/Studienarbeit.git
%cd Studienarbeit/Detection_training

# 1. Setup Paths

In [ ]:
import os
import Tensorflow.scripts.Paths as pp
# %pip install ultralytics
from ultralytics import YOLO

paths = pp.paths
paths.setup_paths()
CUSTOM_MODEL_NAME = 'raccoon_yolov8n_320_B16_ep34'
YOLO_WEIGHTS = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, "weights")
YOLO_BEST = os.path.join(YOLO_WEIGHTS, 'best_saved_model')

# 2. Download pretrained Tensorflow Model and install TFOD

# Make the yaml file

In [ ]:
with open(pp.YOLO_CONFIG_PATH, "w") as config:
    config.write("train: ../trainset/images\n")
    config.write("test: ../testset/images\n")
    config.write("val: ../devset/images\n\n")
    config.write('nc: {}\n'.format(len(pp.LABELS)))
    config.write("names:\n")
    for i in range(0, len(pp.LABELS)):
        label = pp.LABELS[i]
        yaml_name = '   {}: {}'.format(label['id'] - 1, label['name'])
        if i < len(pp.LABELS) - 1:
            config.write(f'{yaml_name}, \n')
        else:
            config.write(f'{yaml_name}\n')


Install TFOD (TensorFlow Object Detection)

# 4. Upload dataset to Colab

Upload the dataset to google drive

In [ ]:
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/Studienarbeit'

if os.name =='posix':
    from google.colab import drive

    drive.mount('/content/drive')

    import shutil
    shutil.copy(os.path.join(GOOGLE_DRIVE_PATH, 'yolo_' + pp.DATASET_NAME), paths.YOLO_IMG_PATH)

In [ ]:
if os.path.exists(pp.YOLO_DATASET):
  !tar -zxvf {pp.YOLO_DATASET}

# For Yolov8

In [ ]:
# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch

# Use the model
results = model.train(
    data=pp.YOLO_CONFIG_PATH,
    epochs=20,
    imgsz=320,
    batch=16,
    name=CUSTOM_MODEL_NAME)  # train the model

In [ ]:
results = model.val()  # evaluate model performance on the validation set

# Test detections on images

The loss metrics and other stuff is already saved in the zip file as .png

In [ ]:
# Load the trained model
model = YOLO(os.path.join(YOLO_WEIGHTS, "best.pt"), task='detect')

In [ ]:
# Validate model locally
results = model.val(data=pp.YOLO_CONFIG_PATH,)  # evaluate model performance on the validation set

In [ ]:
# Run on a few pictures and see how it goes.
results = model(source=os.path.join(paths.DEVSET_PATH), show=True, conf=0.3, save=True, task='detect')

# for later analyzing of performance
%%capture cap --no-stderr
print(f'Raw model {CUSTOM_MODEL_NAME} before converting time interference')
with open('runs/raw_model_perf.txt', 'w') as f:
    f.write(cap.stdout)

## Convert to tflite and TPU

In [ ]:
# it automatically converts for anything you need
# Known Bug all quantization .tflite files are not working (no Detections) same goes with the edgetpu
tflite = model.export(format='edgetpu', imgsz=320)

In [ ]:
# now interfere the tflite file
# edgetpu obviously cannot be interfered without an edgetpu so we will use the last quantization
# Known Bug all quantization
TFLITE_MODEL = os.path.join(YOLO_BEST, 'best_full_integer_quant.tflite')

import tensorflow as tf
interpreter = tf.lite.Interpreter(model_path = TFLITE_MODEL)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

# tflite = YOLO(TFLITE_MODEL, task='detect')

# results = tflite(source=os.path.join(paths.DEVSET_PATH), imgsz=(320,320), conf=0.3, save=True)
# print(results)

# for later analyzing of performance
%%capture cap --no-stderr
print(f'Raw model {CUSTOM_MODEL_NAME} after converting time interference')
with open('runs/quant_model_perf.txt', 'w') as f:
    f.write(cap.stdout)

# Yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!cd yolov5 && pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()

In [ ]:
# train it
command = f'python train.py\
    --img 320 \
    --batch 16 \
    --epochs 20 \
    --data {pp.YOLO_CONFIG_PATH} \
    --weights yolo5n.pt \
    --cache ram'
!{command}

In [ ]:
# Validate it
command = f'python val.py \
   --weight {os.path.join(YOLO_WEIGHTS, "best.pt")} \
   --data {pp.YOLO_CONFIG_PATH} \
   --imgsz 320'

In [ ]:
# export it to edgetpu hopefully
command = f'python export.py\
    --img 320 \
    --weights {YOLO_BEST} \
    --include edgetpu'
!{command}

In [ ]:
# validate that
import torch

# Model
model = torch.hub.load('ultralytics/yolov5',                            'custom',                                                 'yolov5s_edgetpu.tflite')

# Images
img = 'https://ultralytics.com/images/zidane.jpg'  # or file, Path, PIL, OpenCV, numpy, list

# Inference
results = model(img)

# Results
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

# Zip it and save it in Google Drive

In [ ]:
import shutil
shutil.make_archive(os.path.join(GOOGLE_DRIVE_PATH, CUSTOM_MODEL_NAME), 'zip', "runs")